In [1]:
import boto3
import sys
from botocore.exceptions import ClientError
import logging
from time import sleep
import math
import numpy
import datetime
from dateutil.tz import tzlocal

In [2]:
ec2_client = boto3.client('ec2')
response = ec2_client.describe_instances()
response

{'Reservations': [{'Groups': [],
   'Instances': [{'AmiLaunchIndex': 2,
     'ImageId': 'ami-026dea5602e368e96',
     'InstanceId': 'i-01bc49ae38883ba25',
     'InstanceType': 't2.micro',
     'KeyName': 'key',
     'LaunchTime': datetime.datetime(2020, 6, 6, 20, 1, 26, tzinfo=tzutc()),
     'Monitoring': {'State': 'disabled'},
     'Placement': {'AvailabilityZone': 'us-east-2b',
      'GroupName': '',
      'Tenancy': 'default'},
     'PrivateDnsName': 'ip-172-31-23-159.us-east-2.compute.internal',
     'PrivateIpAddress': '172.31.23.159',
     'ProductCodes': [],
     'PublicDnsName': 'ec2-18-218-145-173.us-east-2.compute.amazonaws.com',
     'PublicIpAddress': '18.218.145.173',
     'State': {'Code': 16, 'Name': 'running'},
     'StateTransitionReason': '',
     'SubnetId': 'subnet-4c4a2836',
     'VpcId': 'vpc-2b6c6043',
     'Architecture': 'x86_64',
     'BlockDeviceMappings': [{'DeviceName': '/dev/xvda',
       'Ebs': {'AttachTime': datetime.datetime(2020, 6, 6, 20, 1, 27, tzinf

In [ ]:
#I used https://github.com/awsdocs/aws-doc-sdk-examples/blob/master/python/example_code/ec2/create_instance.py for help here

def create_ec2_instance(image_id, instance_type, keypair_name, security_group, security_group_id, iam_role_name):
    """Provision and launch an EC2 instance
    The method returns without waiting for the instance to reach
    a running state.
    :param image_id: ID of AMI to launch, such as 'ami-XXXX'
    :param instance_type: string, such as 't2.micro'
    :param keypair_name: string, name of the key pair
    :return Dictionary containing information about the instance. If error,
    returns None.
    """

    # Provision and launch the EC2 instance
    ec2_client = boto3.client('ec2')
    try:
        response = ec2_client.run_instances(ImageId=image_id,
                                            InstanceType=instance_type,
                                            KeyName=keypair_name,
                                            SecurityGroupIds = [security_group_id],
                                            SecurityGroups= [security_group],
                                            IamInstanceProfile={
                                                #'Arn': 'arn:aws:iam::578971879148:instance-profile/EnablesEC2ToAccessSystemsManagerRole',
                                                'Name': iam_role_name
                                            },
                                            MinCount=1,
                                            MaxCount=14)
    except ClientError as e:
        logging.error(e)
        return None
    return response


AMI_IMAGE_ID = 'ami-02f53f5f90a9cc773' #This is the Amazon machine image ID
INSTANCE_TYPE = 't2.micro' #this is the type of vm ec2 instance
KEYPAIR_NAME = 'key' #this is the name of the key that allows you to ssh into the instance
SECURITY_GROUP = 'kademlia-all-access' #this is the name of the security group. I custom defined a security group that allows any protocol to connect on any port.
SECURITY_GROUP_ID = 'sg-06474ad72b0f3fd58' #this is the id associated with the security group on our account
IAM_ROLE_NAME = 'EnablesEC2ToAccessSystemsManagerRole' #this is the iam role to allow ssm to send commands to each instance.

# Set up logging
logging.basicConfig(level=logging.DEBUG,
                    format='%(levelname)s: %(asctime)s: %(message)s')

response = create_ec2_instance(AMI_IMAGE_ID, INSTANCE_TYPE, KEYPAIR_NAME, SECURITY_GROUP, SECURITY_GROUP_ID, IAM_ROLE_NAME)

ec2 = boto3.resource('ec2')

# AWS Instance Type Docs: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html#instance
instances = [] #this is a python list of type ec2 instance objects

for instance in response['Instances']:
    logging.info(f'Launched EC2 Instance {instance["InstanceId"]}')
    logging.info(f'    VPC ID: {instance["VpcId"]}')
    logging.info(f'    Private IP Address: {instance["PrivateIpAddress"]}')
    logging.info(f'    Current State: {instance["State"]["Name"]}')
    print(instance)
    instances.append(ec2.Instance(instance["InstanceId"])) #creates the ec2 instance object

for instance in instances:
    instance.wait_until_running()
    print(instance)
    print(instance.public_ip_address)
                  



In [4]:
def get_running_instances():
    """ Returns a list of instance objects for instances that are already running. 
        Does not spawn any instances. 
    """

    ec2_client = boto3.client('ec2')
    response = ec2_client.describe_instances()

    ec2 = boto3.resource('ec2')
    instances = []

    for reservation in response['Reservations']:
        for instance in reservation["Instances"]:
            if instance['State']['Name'] != 'terminated': #we don't want to include the terminated instances
                instances.append(ec2.Instance(instance["InstanceId"])) #creates the ec2 instance object
    
    for instance in instances:
        instance.wait_until_running #this assumes all instances are running, TODO maybe check and then start if not?
        print(instance)
        print(instance.public_ip_address)
    
    return instances

instances = get_running_instances()
instances

ec2.Instance(id='i-01bc49ae38883ba25')
18.218.145.173
ec2.Instance(id='i-053cb74b90426da70')
13.58.116.51
ec2.Instance(id='i-0967bbf8f4155db90')
18.191.111.142
ec2.Instance(id='i-05502eb50803bc085')
18.224.44.242
ec2.Instance(id='i-0528cafad0f7a83d1')
13.59.53.56
ec2.Instance(id='i-054b1ce206e7145dd')
18.217.22.130
ec2.Instance(id='i-002acec677ae5d48a')
18.216.148.64
ec2.Instance(id='i-0944bfc515c84de49')
3.15.194.124
ec2.Instance(id='i-072bfaa26e2a4f1bf')
18.224.15.189
ec2.Instance(id='i-010cc9a960d4b47a6')
18.191.170.156
ec2.Instance(id='i-0306b41039ff6c722')
18.218.168.224
ec2.Instance(id='i-074ae624baf2ef842')
18.191.83.232
ec2.Instance(id='i-00e05716f5c59e4fe')
18.222.136.223
ec2.Instance(id='i-06ca0f848d1535e8f')
3.133.93.43
ec2.Instance(id='i-0a79419577ed911f1')
18.220.114.122
ec2.Instance(id='i-02de60b4166bfa219')
3.135.9.55
ec2.Instance(id='i-0f36a01a9ed725f2e')
3.19.123.251
ec2.Instance(id='i-05bd66b759672afde')
3.20.203.249
ec2.Instance(id='i-06cd04513e420ee2c')
18.191.232.1

[ec2.Instance(id='i-01bc49ae38883ba25'),
 ec2.Instance(id='i-053cb74b90426da70'),
 ec2.Instance(id='i-0967bbf8f4155db90'),
 ec2.Instance(id='i-05502eb50803bc085'),
 ec2.Instance(id='i-0528cafad0f7a83d1'),
 ec2.Instance(id='i-054b1ce206e7145dd'),
 ec2.Instance(id='i-002acec677ae5d48a'),
 ec2.Instance(id='i-0944bfc515c84de49'),
 ec2.Instance(id='i-072bfaa26e2a4f1bf'),
 ec2.Instance(id='i-010cc9a960d4b47a6'),
 ec2.Instance(id='i-0306b41039ff6c722'),
 ec2.Instance(id='i-074ae624baf2ef842'),
 ec2.Instance(id='i-00e05716f5c59e4fe'),
 ec2.Instance(id='i-06ca0f848d1535e8f'),
 ec2.Instance(id='i-0a79419577ed911f1'),
 ec2.Instance(id='i-02de60b4166bfa219'),
 ec2.Instance(id='i-0f36a01a9ed725f2e'),
 ec2.Instance(id='i-05bd66b759672afde'),
 ec2.Instance(id='i-06cd04513e420ee2c'),
 ec2.Instance(id='i-0b7978c54f87d6777')]

In [5]:
# I used https://stackoverflow.com/questions/42645196/how-to-ssh-and-run-commands-in-ec2-using-boto3 for help
# AWS SSM Docs: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ssm.html
# AWS SSM Info: https://docs.aws.amazon.com/systems-manager/latest/userguide/what-is-systems-manager.html
# AWS SSM Getting Started Guide: https://aws.amazon.com/getting-started/hands-on/remotely-run-commands-ec2-instance-systems-manager/

ssm_client = boto3.client('ssm')
ssm_client.describe_instance_information()

{'InstanceInformationList': [{'InstanceId': 'i-0a79419577ed911f1',
   'PingStatus': 'Online',
   'LastPingDateTime': datetime.datetime(2020, 6, 10, 14, 35, 58, 608000, tzinfo=tzlocal()),
   'AgentVersion': '2.3.1319.0',
   'IsLatestVersion': True,
   'PlatformType': 'Linux',
   'PlatformName': 'Amazon Linux',
   'PlatformVersion': '2',
   'ResourceType': 'EC2Instance',
   'IPAddress': '172.31.21.139',
   'ComputerName': 'ip-172-31-21-139.us-east-2.compute.internal'},
  {'InstanceId': 'i-0306b41039ff6c722',
   'PingStatus': 'Online',
   'LastPingDateTime': datetime.datetime(2020, 6, 10, 14, 35, 48, 628000, tzinfo=tzlocal()),
   'AgentVersion': '2.3.1319.0',
   'IsLatestVersion': True,
   'PlatformType': 'Linux',
   'PlatformName': 'Amazon Linux',
   'PlatformVersion': '2',
   'ResourceType': 'EC2Instance',
   'IPAddress': '172.31.27.208',
   'ComputerName': 'ip-172-31-27-208.us-east-2.compute.internal'},
  {'InstanceId': 'i-06cd04513e420ee2c',
   'PingStatus': 'Online',
   'LastPingDate

In [6]:
def cancel_command(command_id):
    remaining_uncanceled = True    
    while(remaining_uncanceled):
        remaining_uncanceled = False
        
        response = ssm_client.list_commands(CommandId = command_id)
        
        for command in response['Commands']: 
            if command['Status'] == 'InProgress' or command['Status'] == 'Pending': 
                remaining_uncanceled = True
                
                #this does not guarantee a command will be cancelled so we must double check or send multiple requests 
                ssm_client.cancel_command(CommandId=command['CommandId'])

In [7]:


def cancel_all_commands(): 
    """
        This function cancels any pending (still running) SSM commands. 
        It enables us to start fresh. 
    """
    
    remaining_uncanceled = True    
    while(remaining_uncanceled):
        remaining_uncanceled = False
        
        response = ssm_client.list_commands()
        
        for command in response['Commands']: 
            if command['Status'] == 'InProgress' or command['Status'] == 'Pending': 
                remaining_uncanceled = True
                
                print(command)
                #this does not guarantee a command will be cancelled so we must double check or send multiple requests 
                ssm_client.cancel_command(CommandId=command['CommandId'])
                
cancel_all_commands()

In [8]:
instances
#instances[0].id

[ec2.Instance(id='i-01bc49ae38883ba25'),
 ec2.Instance(id='i-053cb74b90426da70'),
 ec2.Instance(id='i-0967bbf8f4155db90'),
 ec2.Instance(id='i-05502eb50803bc085'),
 ec2.Instance(id='i-0528cafad0f7a83d1'),
 ec2.Instance(id='i-054b1ce206e7145dd'),
 ec2.Instance(id='i-002acec677ae5d48a'),
 ec2.Instance(id='i-0944bfc515c84de49'),
 ec2.Instance(id='i-072bfaa26e2a4f1bf'),
 ec2.Instance(id='i-010cc9a960d4b47a6'),
 ec2.Instance(id='i-0306b41039ff6c722'),
 ec2.Instance(id='i-074ae624baf2ef842'),
 ec2.Instance(id='i-00e05716f5c59e4fe'),
 ec2.Instance(id='i-06ca0f848d1535e8f'),
 ec2.Instance(id='i-0a79419577ed911f1'),
 ec2.Instance(id='i-02de60b4166bfa219'),
 ec2.Instance(id='i-0f36a01a9ed725f2e'),
 ec2.Instance(id='i-05bd66b759672afde'),
 ec2.Instance(id='i-06cd04513e420ee2c'),
 ec2.Instance(id='i-0b7978c54f87d6777')]

In [9]:
# def execute_commands_on_linux_instances(client, commands, instance_ids):
#     """Runs commands on remote linux instances
#     :param client: a boto/boto3 ssm client
#     :param commands: a list of strings, each one a command to execute on the instances
#     :param instance_ids: a list of instance_id strings, of the instances on which to execute the command
#     :return: the response from the send_command function (check the boto3 docs for ssm client.send_command() )
#     """

#     resp = client.send_command(
#         DocumentName="AWS-RunShellScript", # One of AWS' preconfigured documents
#         Parameters={'commands': commands},
#         InstanceIds=instance_ids,
#     )
#     return resp

# # Example use:
# # commands = ['python3 /home/ec2-user/CS244B/aws/kademlia_scripts/first_node.py 20 3']
# # instance_ids = ['i-01bc49ae38883ba25']
# # response1 = execute_commands_on_linux_instances(ssm_client, commands, instance_ids)
# # response1

# # commands = ['python3 /home/ec2-user/CS244B/aws/kademlia_scripts/test.py 18.218.145.173 8468 a 5']
# # instance_ids = ['i-029096fe3848618fe']
# # response2 = execute_commands_on_linux_instances(ssm_client, commands, instance_ids)
# # response2

# # ssm_client.get_command_invocation(CommandId=response1['Command']['CommandId'],
# #     InstanceId= response1['Command']['InstanceIds'][0])

# # ssm_client.get_command_invocation(CommandId=response2['Command']['CommandId'],
# #     InstanceId= response2['Command']['InstanceIds'][0])

In [ ]:
def run_nodes(ksize, alpha, instances, mean_time=100):
    ''' This function will run the actual testing for success rate once
        we have all the instances spawned and set up. It assumes instances[0]
        is the first (bootstrapping) node that never churns. This is desgined to 
        be defined within another function or in a larger script.
        
        @mean_time is how long each server instance should be alive on 
        average. The same value is used for how long a server is down when 
        it fails. The mean uptime and downtime and therefore the same. 
    '''
    
    def execute_commands_on_linux_instances(client, commands, instance_ids):
        """Runs commands on remote linux instances
        :param client: a boto/boto3 ssm client
        :param commands: a list of strings, each one a command to execute on the instances
        :param instance_ids: a list of instance_id strings, of the instances on which to execute the command
        :return: the response from the send_command function (check the boto3 docs for ssm client.send_command() )
        """

        resp = client.send_command(
            DocumentName="AWS-RunShellScript", # One of AWS' preconfigured documents
            Parameters={'commands': commands},
            InstanceIds=instance_ids,
        )
        return resp
    
    first_node_ip = instances[0].public_ip_address
    if(first_node_ip == None): 
        println("\n\n\n\n\n ERROR: Failed to get public IPv4 address for the first (bootstrapping) node")
    
    first_node_command = 'python3 /home/ec2-user/CS244B/aws/kademlia_scripts/first_node.py {} {}'.format(ksize, alpha) 
    bootstrap_server_command = 'python3 /home/ec2-user/CS244B/aws/kademlia_scripts/run_node.py {} 8468 {} {}'.format(first_node_ip, ksize, alpha)
    
    def get_command(key): 
        _get_command = 'python3 /home/ec2-user/CS244B/aws/kademlia_scripts/get.py {} 8468 {}'.format(first_node_ip, key)
        return _get_command
    def set_command(key, value):
        _set_command = 'python3 /home/ec2-user/CS244B/aws/kademlia_scripts/set.py {} 8468 {} {}'.format(first_node_ip, key, value)
        return _set_command
    def sleep_command(time):
        _sleep_command = 'sleep {}'.format(time)
        return _sleep_command

    set_instance_index = len(instances)-2
    get_instance_index = len(instances)-1 
    
    # this will be used to set values and determine whether we got the right value back or not 
    truth_table = {
        'a':1, 
        'b':2,
        'c':3,
        'd':4,
        'e':5,
        'f':6,
        'g':7,
        'h':8,
        'i':9,
        'j':10,
        'k':11,
        'l':12,
        'm':13,
        'n':14,
        'o':15,
        'p':16,
        'q':17,
        'r':18,
        's':19,
        't':20,
        'u':21,
        'v':22,
        'w':23,
        'x':24,
        'y':25,
        'z':26,
    }
    
    # note the assigned timeout is not used for the first instance, which  always remains live 
    # or for the last two instances which are used to get and set nodes and do not run long lived servers
    timeouts = [numpy.random.exponential(mean_time) for i in range(len(instances))]
    
    command_responses = ["" for i in range(len(instances))]
    
    #start first node. The first node is the bootstrapping node that does not churn 
    commands = [first_node_command]
    instance_ids = [instances[0].id]
    command_responses[0] = execute_commands_on_linux_instances(ssm_client, commands, instance_ids)    
    sleep(3)
    
    # start the kademlia servers, these nodes will churn 
    for index in range(1, len(instances)-2): # we leave the final two instances for set and get requests 
        
        commands = [bootstrap_server_command]
        instance_ids = [instances[index].id]
        command_responses[index] = execute_commands_on_linux_instances(ssm_client, commands, instance_ids)
    
    sleep(10) #allow the instances a chance to connect and populate routing tables
                
    def churn():
        
        def get_elapsed_time(command):
            time_response = ssm_client.list_commands(CommandId = command['CommandId'])
            dt = time_response['Commands'][0]['RequestedDateTime']
            now = datetime.datetime.now(dt.tzinfo)
            return (now - dt).seconds
        
        
        #TODO there is a problem where commands are run on nodes that are already running kademlia 
        for command_idx in range(1, len(command_responses)-2): 
            command = command_responses[command_idx]['Command']
            
            #get updated info on the status of the command
            command_info = ssm_client.get_command_invocation(CommandId=command['CommandId'], 
                                              InstanceId= command['InstanceIds'][0])
            print(command_info)
            
            if get_elapsed_time(command) > timeouts[command_idx]: #simulate node churn
                cancel_command(command['CommandId']) #this is a blocking call and makes sure the command is cancelled
                
                commands = [sleep_command(timeouts[command_idx])]
                instance_ids = [command['InstanceIds'][0]]
                command_responses[command_idx] = execute_commands_on_linux_instances(ssm_client, commands, instance_ids)
                
            elif(command_info['Status'] == 'Success'): # restart kademlia server, simulated downtime has ended
                assert('sleep' in command['Parameters']['commands'][0])
                
                # recalculate the exponentially distributed timeout here
                timeouts[command_idx] = numpy.random.exponential(mean_time)

                #we must restart the kademlia server
                commands = [bootstrap_server_command]
                instance_ids = [command['InstanceIds'][0]]
                command_responses[command_idx] = execute_commands_on_linux_instances(client, commands, instance_ids)
            
            elif command_info['Status'] == 'InProgress' or command_info['Status'] == 'Pending':
                pass
                #this is expected
            else: 
                print("\n\n\n\n\ UNEXPECTED BEHAVIOR: The command didn't succeed or just keep running")
                print(command_info['Status'])
                    
    def evaluate_get_response(get_response, key, value):
        stdout_result = get_response['StandardOutputContent']
        print(stdout_result)
        result_list = stdout_result.split(' ')
        idx = result_list.index('result:')
        result = result_list[idx+1]
        if "None" in result:
            return False
        if str(value) in result:
            return True
        else: 
            print("Inconclusive Result: {} for value {}".format(stdout_result, value))
            return False
        
    def wait_until_complete(command):
        sleep(1)
        
        while(True):
            command_info = ssm_client.get_command_invocation(CommandId=command['Command']['CommandId'], 
                                              InstanceId= command['Command']['InstanceIds'][0])
              
            if(command_info['Status'] == "Success"): 
                return True
            if(command_info['Status'] == "Failed" or command_info['Status'] == "TimedOut"):
                print("Print command we were waiting for failed: {}".format(command_info))
                return False
            else: 
                sleep(1)
            

    success_list = []
    
    for key, value in truth_table.items():
        
        churn()
        
        #begin to make get and set requests to judge the success rate
        commands = [set_command(key, value)]
        instance_ids = [instances[set_instance_index].id] #this will be the set instance index
        command_responses[set_instance_index] = execute_commands_on_linux_instances(ssm_client, commands, instance_ids)
        if not wait_until_complete(command_responses[set_instance_index]):
            #TODO relaunch
            pass
        
        churn()

        commands = [get_command(key)]
        instance_ids = [instances[get_instance_index].id] #this will be the get instance index
        command_responses[get_instance_index] = execute_commands_on_linux_instances(ssm_client, commands, instance_ids)
        if not wait_until_complete(command_responses[get_instance_index]):
            #TODO relaunch
            pass

        get_response = ssm_client.get_command_invocation(CommandId=command_responses[get_instance_index]['Command']['CommandId'], 
                                          InstanceId= command_responses[get_instance_index]['Command']['InstanceIds'][0])
        print(get_response)

        if evaluate_get_response(get_response, key, value):
            success_list.append(1)
        else:
            success_list.append(0)
    
    success_rate = sum(success_list)/len(truth_table)
    return success_rate

            
run_nodes(20, 3, instances)


{'CommandId': '049c4e63-9911-448e-9b91-038489252cd8', 'InstanceId': 'i-053cb74b90426da70', 'Comment': '', 'DocumentName': 'AWS-RunShellScript', 'DocumentVersion': '', 'PluginName': 'aws:runShellScript', 'ResponseCode': -1, 'ExecutionEndDateTime': '', 'Status': 'InProgress', 'StatusDetails': 'InProgress', 'StandardOutputContent': '', 'StandardOutputUrl': '', 'StandardErrorContent': '', 'StandardErrorUrl': '', 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}, 'ResponseMetadata': {'RequestId': '54dfa013-b7e6-48da-a7c7-e11e12ebf820', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '54dfa013-b7e6-48da-a7c7-e11e12ebf820', 'content-type': 'application/x-amz-json-1.1', 'content-length': '471', 'date': 'Wed, 10 Jun 2020 18:37:16 GMT'}, 'RetryAttempts': 0}}




\ UNEXPECTED BEHAVIOR: The command didn't succeed or just keep running
InProgress
{'CommandId': 'eeb81401-208d-4818-9659-56e0edfab68a', 'InstanceId': 'i-0967bbf8f4155db90', 'Comment': '

{'CommandId': 'fbea23fc-1ba1-404a-9ec2-93091f19875d', 'InstanceId': 'i-00e05716f5c59e4fe', 'Comment': '', 'DocumentName': 'AWS-RunShellScript', 'DocumentVersion': '', 'PluginName': 'aws:runShellScript', 'ResponseCode': -1, 'ExecutionEndDateTime': '', 'Status': 'InProgress', 'StatusDetails': 'InProgress', 'StandardOutputContent': '', 'StandardOutputUrl': '', 'StandardErrorContent': '', 'StandardErrorUrl': '', 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}, 'ResponseMetadata': {'RequestId': '72d58bf6-2b81-4f08-84e7-ec859436490c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '72d58bf6-2b81-4f08-84e7-ec859436490c', 'content-type': 'application/x-amz-json-1.1', 'content-length': '471', 'date': 'Wed, 10 Jun 2020 18:37:20 GMT'}, 'RetryAttempts': 0}}




\ UNEXPECTED BEHAVIOR: The command didn't succeed or just keep running
InProgress
{'CommandId': '4b3fae7e-4d20-4f6f-9ded-c3c75ae42945', 'InstanceId': 'i-06ca0f848d1535e8f', 'Comment': '

{'CommandId': '407d3e1f-e6d4-441d-a313-92322cf19468', 'InstanceId': 'i-054b1ce206e7145dd', 'Comment': '', 'DocumentName': 'AWS-RunShellScript', 'DocumentVersion': '', 'PluginName': 'aws:runShellScript', 'ResponseCode': -1, 'ExecutionEndDateTime': '', 'Status': 'InProgress', 'StatusDetails': 'InProgress', 'StandardOutputContent': '', 'StandardOutputUrl': '', 'StandardErrorContent': '', 'StandardErrorUrl': '', 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}, 'ResponseMetadata': {'RequestId': '067e4f52-b2c0-4968-927a-d95109aa8a96', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '067e4f52-b2c0-4968-927a-d95109aa8a96', 'content-type': 'application/x-amz-json-1.1', 'content-length': '471', 'date': 'Wed, 10 Jun 2020 18:37:43 GMT'}, 'RetryAttempts': 0}}




\ UNEXPECTED BEHAVIOR: The command didn't succeed or just keep running
InProgress
{'CommandId': '1de70d3d-c277-428e-aa77-fede95e3d27f', 'InstanceId': 'i-002acec677ae5d48a', 'Comment': '

{'CommandId': '0d498bd4-55ca-4f6e-b4df-72519ee05963', 'InstanceId': 'i-02de60b4166bfa219', 'Comment': '', 'DocumentName': 'AWS-RunShellScript', 'DocumentVersion': '', 'PluginName': 'aws:runShellScript', 'ResponseCode': -1, 'ExecutionEndDateTime': '', 'Status': 'InProgress', 'StatusDetails': 'InProgress', 'StandardOutputContent': '', 'StandardOutputUrl': '', 'StandardErrorContent': '', 'StandardErrorUrl': '', 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}, 'ResponseMetadata': {'RequestId': 'aba32d7d-01a5-442b-9f30-893d9d3f6ffd', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'aba32d7d-01a5-442b-9f30-893d9d3f6ffd', 'content-type': 'application/x-amz-json-1.1', 'content-length': '471', 'date': 'Wed, 10 Jun 2020 18:37:47 GMT'}, 'RetryAttempts': 0}}




\ UNEXPECTED BEHAVIOR: The command didn't succeed or just keep running
InProgress
{'CommandId': '587da576-4f45-42a4-8e3b-488e30b25ae5', 'InstanceId': 'i-0f36a01a9ed725f2e', 'Comment': '

{'CommandId': 'eeb81401-208d-4818-9659-56e0edfab68a', 'InstanceId': 'i-0967bbf8f4155db90', 'Comment': '', 'DocumentName': 'AWS-RunShellScript', 'DocumentVersion': '', 'PluginName': 'aws:runShellScript', 'ResponseCode': -1, 'ExecutionEndDateTime': '', 'Status': 'InProgress', 'StatusDetails': 'InProgress', 'StandardOutputContent': '', 'StandardOutputUrl': '', 'StandardErrorContent': '', 'StandardErrorUrl': '', 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}, 'ResponseMetadata': {'RequestId': 'c7ee4ade-3ac0-4b70-adfc-c451e850ff4c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c7ee4ade-3ac0-4b70-adfc-c451e850ff4c', 'content-type': 'application/x-amz-json-1.1', 'content-length': '471', 'date': 'Wed, 10 Jun 2020 18:38:17 GMT'}, 'RetryAttempts': 0}}
{'CommandId': 'df8b5b02-f9aa-4863-adae-e7c956389e3f', 'InstanceId': 'i-05502eb50803bc085', 'Comment': '', 'DocumentName': 'AWS-RunShellScript', 'DocumentVersion': '', 'PluginName': 'aws:run

...

{'CommandId': 'e904f267-7332-464c-846c-be80e508d058', 'InstanceId': 'i-0a79419577ed911f1', 'Comment': '', 'DocumentName': 'AWS-RunShellScript', 'DocumentVersion': '', 'PluginName': 'aws:runShellScript', 'ResponseCode': 0, 'ExecutionStartDateTime': '2020-06-10T18:44:30.443Z', 'ExecutionElapsedTime': 'PT26.574S', 'ExecutionEndDateTime': '2020-06-10T18:44:56.443Z', 'Status': 'Success', 'StatusDetails': 'Success', 'StandardOutputContent': '', 'StandardOutputUrl': '', 'StandardErrorContent': '', 'StandardErrorUrl': '', 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}, 'ResponseMetadata': {'RequestId': 'db507cb6-052a-4219-8984-8a744eb9edff', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'db507cb6-052a-4219-8984-8a744eb9edff', 'content-type': 'application/x-amz-json-1.1', 'content-length': '575', 'date': 'Wed, 10 Jun 2020 18:45:47 GMT'}, 'RetryAttempts': 0}}
{'CommandId': '88dcf3dc-7b05-48c9-b73f-038056c73c04', 'InstanceId': 'i-02de60b416

{'CommandId': 'f698b9df-b637-439e-9bf1-b6d387fbdc02', 'InstanceId': 'i-05502eb50803bc085', 'Comment': '', 'DocumentName': 'AWS-RunShellScript', 'DocumentVersion': '', 'PluginName': 'aws:runShellScript', 'ResponseCode': -1, 'ExecutionEndDateTime': '', 'Status': 'InProgress', 'StatusDetails': 'InProgress', 'StandardOutputContent': '', 'StandardOutputUrl': '', 'StandardErrorContent': '', 'StandardErrorUrl': '', 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}, 'ResponseMetadata': {'RequestId': '742b4ba6-1bf0-4f88-bad7-e312c18a8665', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '742b4ba6-1bf0-4f88-bad7-e312c18a8665', 'content-type': 'application/x-amz-json-1.1', 'content-length': '471', 'date': 'Wed, 10 Jun 2020 18:46:25 GMT'}, 'RetryAttempts': 0}}




\ UNEXPECTED BEHAVIOR: The command didn't succeed or just keep running
InProgress
{'CommandId': 'aea475ad-e9a6-4ab9-9985-a0f45f38b355', 'InstanceId': 'i-0528cafad0f7a83d1', 'Comment': '

{'CommandId': '2e98c51a-3b83-486d-83fc-b8e89fcf03f6', 'InstanceId': 'i-00e05716f5c59e4fe', 'Comment': '', 'DocumentName': 'AWS-RunShellScript', 'DocumentVersion': '', 'PluginName': 'aws:runShellScript', 'ResponseCode': -1, 'ExecutionEndDateTime': '', 'Status': 'InProgress', 'StatusDetails': 'InProgress', 'StandardOutputContent': '', 'StandardOutputUrl': '', 'StandardErrorContent': '', 'StandardErrorUrl': '', 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}, 'ResponseMetadata': {'RequestId': '3aea42fb-c216-4dd6-9e57-4d84e270db32', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '3aea42fb-c216-4dd6-9e57-4d84e270db32', 'content-type': 'application/x-amz-json-1.1', 'content-length': '471', 'date': 'Wed, 10 Jun 2020 18:46:27 GMT'}, 'RetryAttempts': 0}}




\ UNEXPECTED BEHAVIOR: The command didn't succeed or just keep running
InProgress
{'CommandId': 'ee84d10c-f74d-400e-bf64-3fc0cd593300', 'InstanceId': 'i-06ca0f848d1535e8f', 'Comment': '

{'CommandId': '7eae00c8-677c-4cdd-87ff-6094f7a51923', 'InstanceId': 'i-0528cafad0f7a83d1', 'Comment': '', 'DocumentName': 'AWS-RunShellScript', 'DocumentVersion': '', 'PluginName': 'aws:runShellScript', 'ResponseCode': 0, 'ExecutionStartDateTime': '2020-06-10T18:46:25.837Z', 'ExecutionElapsedTime': 'PT7.012S', 'ExecutionEndDateTime': '2020-06-10T18:46:32.837Z', 'Status': 'Success', 'StatusDetails': 'Success', 'StandardOutputContent': '', 'StandardOutputUrl': '', 'StandardErrorContent': '', 'StandardErrorUrl': '', 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}, 'ResponseMetadata': {'RequestId': 'd561d137-8e30-4646-a3d8-f607b87ef8a4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd561d137-8e30-4646-a3d8-f607b87ef8a4', 'content-type': 'application/x-amz-json-1.1', 'content-length': '574', 'date': 'Wed, 10 Jun 2020 18:47:44 GMT'}, 'RetryAttempts': 0}}
{'CommandId': '72b9ca65-57b5-44b9-98dd-a428edd69aa0', 'InstanceId': 'i-054b1ce206e

{'CommandId': 'a2e4525e-6918-435e-b5da-aa9286b5b36f', 'InstanceId': 'i-0a79419577ed911f1', 'Comment': '', 'DocumentName': 'AWS-RunShellScript', 'DocumentVersion': '', 'PluginName': 'aws:runShellScript', 'ResponseCode': 0, 'ExecutionStartDateTime': '2020-06-10T18:46:31.286Z', 'ExecutionElapsedTime': 'PT26.574S', 'ExecutionEndDateTime': '2020-06-10T18:46:57.286Z', 'Status': 'Success', 'StatusDetails': 'Success', 'StandardOutputContent': '', 'StandardOutputUrl': '', 'StandardErrorContent': '', 'StandardErrorUrl': '', 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}, 'ResponseMetadata': {'RequestId': '0d32e0c4-c8b8-474a-afff-e6d56db5a68c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0d32e0c4-c8b8-474a-afff-e6d56db5a68c', 'content-type': 'application/x-amz-json-1.1', 'content-length': '575', 'date': 'Wed, 10 Jun 2020 18:47:49 GMT'}, 'RetryAttempts': 0}}
{'CommandId': '650259ba-a7fa-4d38-ab21-fc59e7ad3d4e', 'InstanceId': 'i-02de60b416

In [ ]:
success_rate = run_nodes(20, 3, instances, meantime=200)

In [ ]:
ssm_client.get_command_invocation(CommandId='5dc48e96-3268-42c6-9840-f7cd70048833',
    InstanceId='i-0967bbf8f4155db90' )

In [ ]:
# # #begin to make get and set requests to judge the success rate
# # commands = [set_command('a', '1')]
# # instance_ids = [instances[set_instance_index].id] #this will be the set instance index
# # command_responses[set_instance_index] = execute_commands_on_linux_instances(ssm_client, commands, instance_ids)

# # commands = [get_command('a')]
# # instance_ids = [instances[get_instance_index].id] #this will be the get instance index
# # command_responses[get_instance_index] = execute_commands_on_linux_instances(ssm_client, commands, instance_ids)

# ssm_client.get_command_invocation(CommandId=responses[10]['Command']['CommandId'],
#     InstanceId= responses[10]['Command']['InstanceIds'][0])

In [ ]:
len(ssm_client.list_commands()['Commands'])

In [ ]:
# ssm_client.list_commands(InstanceId='i-01bc49ae38883ba25')
ssm_client.list_commands()

In [ ]:
ssm_client.get_command_invocation(CommandId='1e7c3e78-2793-4f0c-b6ee-bb0f22d6f124',
    InstanceId='i-05502eb50803bc085' )